### Импорт библиотек

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import mne
import warnings
import torch
import torch.nn as nn
import torch.optim as optim

from tqdm import tqdm

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

from sklearn.decomposition import PCA

from pyriemann.estimation import Covariances
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#%load_ext autoreload
#%autoreload 2
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#!pip install mne
#!pip install pyriemann

In [6]:
#!pipreqs . --force

## 1) Получаем набор данных и проводим аугментацию

### eyes open/eyes closed on EEG Motor Movement/Imagery Dataset

- EEG Motor Movement датасет состоит из 109 испытуемых
- Данные каждого испытуемого состоят из 14 выполненных тестов
- Мы сейчас сосредоточимся за заданиях 1 и 2 с классификацией открытых/закрытых глаз
- Задание 1 $-$ бег с открытыми глазами
- Задание 2 $-$ бег с закрытыми глазами

Subject_1

- Число каналов 64
- Размер выборки 9760
- Частота измерений 160 $c^{-1}$

In [5]:
class data():
    def __init__(self):
        pass

    def get_data(self):
        data = []

        for i in range(109):
            if len(str(i+1)) == 1:
                sub_number = '00'+str(i+1)
            elif len(str(i+1)) == 2:
                sub_number = '0'+str(i+1)
            else:
                sub_number = str(i+1)

            #path_0 = os.path.join(os.path.dirname(os.getcwd()), 'code', 'eeg-motor-movementimagery-dataset', 'files', 'S' + sub_number, 'S' + sub_number + 'R01.edf')
            #path_1 = os.path.join(os.path.dirname(os.getcwd()), 'code', 'eeg-motor-movementimagery-dataset', 'files', 'S' + sub_number, 'S' + sub_number + 'R02.edf')

            # colab
            path_0 = os.path.join('/content/drive/My Drive/7_sem/diplom/', 'code', 'eeg-motor-movementimagery-dataset', 'files', 'S' + sub_number, 'S' + sub_number + 'R01.edf')
            path_1 = os.path.join('/content/drive/My Drive/7_sem/diplom/', 'code', 'eeg-motor-movementimagery-dataset', 'files', 'S' + sub_number, 'S' + sub_number + 'R02.edf')


            X_0 = mne.io.read_raw_edf(path_0, preload=True, verbose = False).get_data()
            X_1 = mne.io.read_raw_edf(path_1, preload=True, verbose = False).get_data()
            data.append((X_0, 0))
            data.append((X_1, 1))
        self.data = np.array(data)

    # Аугментация
    def get_augmented_data(self, window_size = 610):
        self.get_data()
        augmented_data = []
        for time_series, label in self.data:
            divided_time_series = [time_series[:, i:i+window_size] for i in range(0, time_series.shape[1], window_size)]
            for ts in divided_time_series:
                augmented_data.append((ts, label))

        self.augmented_data = np.array(augmented_data)

        # Приводим к одному размеру окна
        new_augmented_data = []
        for i in range(len(self.augmented_data)):
            if self.augmented_data[i][0].shape[1] == 610:
                new_augmented_data.append(self.augmented_data[i])

        self.augmented_data = np.array(new_augmented_data)

`TODO Стоит добавить еще аугментаций и изучить методы аугментаций временных рядов`

In [6]:
dataset = data()
dataset.get_augmented_data()
augmented_data = dataset.augmented_data

In [7]:
augmented_data.shape

(3483, 2)

In [8]:
ts = np.array([augmented_data[i][0] for i in range(len(augmented_data))])
y = np.array([augmented_data[i][1] for i in range(len(augmented_data))])

### Разделим данные на тренировочные и тестовые

In [9]:
ts_train, ts_test, y_train, y_test = train_test_split(ts, y, test_size=0.2, random_state=42)

In [10]:
print(f'In train data class 0: {np.sum(1 - y_train)}, class 1: {np.sum(y_train)}')

In train data class 0: 1392, class 1: 1394


## Воспользуемся библиотекой pyriemann

### Перейдем в касательное пространство и воспользуемся классическими методами классификации

In [11]:
ts_train.shape # Ntrials x Nchannels X Nsamples

(2786, 64, 610)

In [12]:
F1_score = {}
Accuracy = {}

### 1) SVM

In [66]:
# build pipeline
covest = Covariances()
ts = TangentSpace()
svc = SVC(kernel='rbf')

clf = make_pipeline(covest,ts,svc)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)

print(accuracy.mean())

0.9393377219230002


In [67]:
clf = make_pipeline(covest,ts,svc)
clf.fit(ts_train, y_train)

y_pred = clf.predict(ts_test)

In [68]:
print(f'Accuracy on test {round(accuracy_score(y_test, y_pred), 3)}, f1-score on test {round(f1_score(y_test, y_pred), 3)}')

Accuracy['SVM'] = round(accuracy_score(y_test, y_pred), 3)
F1_score['SVM'] = round(f1_score(y_test, y_pred), 3)

Accuracy on test 0.954, f1-score on test 0.954


Аналогичным образом попробуем другие классические методы классификации после переход в касательное пространство

### 2) LogisticRegression

In [69]:
covest = Covariances()
ts = TangentSpace()
logreg = LogisticRegression()

clf = make_pipeline(covest,ts, logreg)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)

print(accuracy.mean())

0.913853014420571


In [70]:
clf = make_pipeline(covest,ts,logreg)
clf.fit(ts_train, y_train)

y_pred = clf.predict(ts_test)

In [71]:
print(f'Accuracy on test {round(accuracy_score(y_test, y_pred), 3)}, f1-score on test {round(f1_score(y_test, y_pred), 3)}')

Accuracy['LogisticRegression'] = round(accuracy_score(y_test, y_pred), 3)
F1_score['LogisticRegression'] = round(f1_score(y_test, y_pred), 3)

Accuracy on test 0.91, f1-score on test 0.909


### 3) DecisionTreeClassifier

In [72]:
covest = Covariances()
ts = TangentSpace()
rf= RandomForestClassifier(n_estimators=200)

clf = make_pipeline(covest,ts, rf)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)
print(accuracy.mean())

0.8797519996396467


In [73]:
clf = make_pipeline(covest,ts,rf)
clf.fit(ts_train, y_train)

y_pred = clf.predict(ts_test)

In [74]:
print(f'Accuracy on test {round(accuracy_score(y_test, y_pred), 3)}, f1-score on test {round(f1_score(y_test, y_pred), 3)}')

Accuracy['RandomForestClassifier'] = round(accuracy_score(y_test, y_pred), 3)
F1_score['RandomForestClassifier'] = round(f1_score(y_test, y_pred), 3)

Accuracy on test 0.9, f1-score on test 0.901


### 4) Добавим фильтрацию XdawnCovariances


In [ ]:
# SVM
covest = XdawnCovariances()
ts = TangentSpace(metric='riemann')
svc = SVC(kernel='rbf')

clf = make_pipeline(covest,ts,svc)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)

print(accuracy.mean())

0.7738647259061923


In [ ]:
# LogisticRegression
covest = covest = XdawnCovariances()
ts = TangentSpace()
logreg = LogisticRegression()

clf = make_pipeline(covest,ts, logreg)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)

print(accuracy.mean())

0.7282909596339839


In [ ]:
# RandomForestClassifier
covest = XdawnCovariances()
ts = TangentSpace()
rf= RandomForestClassifier(n_estimators=200)

clf = make_pipeline(covest,ts, rf)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)
print(accuracy.mean())

0.7447848497133261


### 5) Применим PCA

In [ ]:
# SVM
covest = Covariances()
ts = TangentSpace(metric='riemann')
pca = PCA(1000)
svc = SVC(kernel='rbf')

clf = make_pipeline(covest,ts,pca,svc)
# cross validation
accuracy = cross_val_score(clf, ts_train, y_train)

print(accuracy.mean())

0.9432874526231796


### 6) Полносвязная нейронная сеть

In [13]:
covest = Covariances()
ts = TangentSpace(metric='riemann')
preprocess = make_pipeline(covest,ts)
preprocess.fit(ts_train, y_train)

X_train = preprocess.transform(ts_train)
X_test = preprocess.transform(ts_test)

In [14]:
def get_loader(X, y, batch_size=64):
    train = torch.utils.data.TensorDataset(torch.from_numpy(X).float(),
                                       torch.from_numpy(y).long())
    train_loader = torch.utils.data.DataLoader(train,
                                               batch_size=batch_size)
    return train_loader

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2080, 1040),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Linear(1040, 500),
            nn.ReLU())
        self.layer3 = nn.Sequential(
            nn.Linear(500, 100),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(100, 20),
            nn.ReLU())
        self.layer5 = nn.Linear(20, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        return x

In [60]:
net = Net()

In [61]:
def count_parameters(model):
    return sum(param.data.numpy().size for param \
               in model.parameters() if param.requires_grad)

count_parameters(net)

2736902

In [62]:
criterion = nn.CrossEntropyLoss() # loss includes softmax
optimizer = optim.SGD(net.parameters(), lr=0.01)
#optimizer = optim.Adam(net.parameters(), lr=0.001)

In [63]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

net = net.to(device)

In [64]:
def train_epoch(model, optimizer, train_loader, criterion, device):
    model.train()

    for batch_idx, (data_inputs, data_labels) in enumerate(train_loader):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(data_inputs)

        ## Step 3: Calculate the loss
        loss = criterion(preds, data_labels.long())
        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()

        # Perform backpropagation
        loss.backward()

        ## Step: Update the parameters
        optimizer.step()


def evaluate_loss_acc(loader, model, criterion, device):
    model.eval() # Set model to eval mode
    loss, true_preds, num_preds = 0., 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs, data_labels in loader:

            # Determine prediction of model on dev set
            data_inputs, data_labels = data_inputs.to(device), data_labels.to(device)
            preds = model(data_inputs)

            # Calculate the loss on the batch
            batch_loss = criterion(preds, data_labels.long())
            # Add batch_loss to the summary loss
            loss += batch_loss * data_labels.shape[0]

            preds = torch.softmax(preds, dim = 1) # Softmax to map predictions between 0 and 1
            pred_labels = torch.argmax(preds, dim=1)

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (pred_labels == data_labels).sum()
            num_preds += data_labels.shape[0]

    acc = true_preds / num_preds
    loss = loss / num_preds
    return (loss, acc)
    print(f"Loss of the model: {loss}%")
    print(f"Accuracy of the model: {100.0*acc:4.2f}%")



def train(model, opt, train_loader, test_loader, criterion, n_epochs, \
          device, verbose=True):

    train_log, train_acc_log = [], []
    val_log, val_acc_log = [], []

    for epoch in range(n_epochs):
        train_epoch(model, opt, train_loader, criterion, device)
        train_loss, train_acc = evaluate_loss_acc(train_loader,
                                                  model, criterion,
                                                  device)
        val_loss, val_acc = evaluate_loss_acc(test_loader, model,
                                              criterion, device)

        train_log.append(train_loss.cpu())
        train_acc_log.append(train_acc.cpu())

        val_log.append(val_loss.cpu())
        val_acc_log.append(val_acc.cpu())

        if verbose:
             print (('Epoch [%d/%d], Loss (train/test): %.4f/%.4f,'+\
               ' Acc (train/test): %.4f/%.4f' )
                   %(epoch+1, n_epochs, \
                     train_loss, val_loss, train_acc, val_acc))

    return train_log, train_acc_log, val_log, val_acc_log

In [65]:
n_epochs = 100

train_loader = get_loader(X_train, y_train, batch_size=64)
test_loader = get_loader(X_test, y_test, batch_size=64)

train_log, train_acc_log, test_log, test_acc_log = train(net, optimizer, train_loader, test_loader, criterion, n_epochs, device, verbose=True)

Epoch [1/100], Loss (train/test): 0.6928/0.6928, Acc (train/test): 0.4996/0.4993
Epoch [2/100], Loss (train/test): 0.6926/0.6925, Acc (train/test): 0.4996/0.4993
Epoch [3/100], Loss (train/test): 0.6923/0.6923, Acc (train/test): 0.5022/0.5079
Epoch [4/100], Loss (train/test): 0.6921/0.6921, Acc (train/test): 0.5219/0.5251
Epoch [5/100], Loss (train/test): 0.6918/0.6918, Acc (train/test): 0.5531/0.5538
Epoch [6/100], Loss (train/test): 0.6915/0.6916, Acc (train/test): 0.6106/0.5897
Epoch [7/100], Loss (train/test): 0.6912/0.6913, Acc (train/test): 0.6508/0.6471
Epoch [8/100], Loss (train/test): 0.6909/0.6910, Acc (train/test): 0.6730/0.6657
Epoch [9/100], Loss (train/test): 0.6906/0.6907, Acc (train/test): 0.6866/0.6657
Epoch [10/100], Loss (train/test): 0.6902/0.6904, Acc (train/test): 0.6942/0.6786
Epoch [11/100], Loss (train/test): 0.6898/0.6900, Acc (train/test): 0.7039/0.6729
Epoch [12/100], Loss (train/test): 0.6894/0.6896, Acc (train/test): 0.7114/0.6858
Epoch [13/100], Loss (tra

----

In [ ]:
from models import EEGNet
from fitting import train

if torch.cuda.is_available():
    target_device = 'cuda'
else:
    target_device = 'cpu'

model = EEGNet().to(target_device)